<a href="https://colab.research.google.com/github/OmarAdlii/LLM-Web-Search-Rag-Application/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**APP**

In [1]:
!pip install streamlit crawl4ai markdown ollama chromadb ddgs langchain-community unstructured pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of ddgs to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 58.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh
# Start Ollama
!nohup ollama serve > /tmp/ollama.log 2>&1 &

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!ngrok config add-authtoken 31p71nHtXUDA3X1eEN7tcQAJAqL_7Nhpezx4Yh6NiAAB6GjNx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!playwright install

172.5 MiB [] 0% 0.0s172.5 MiB [] 0% 179.6s172.5 MiB [] 0% 188.3s172.5 MiB [] 0% 124.1s172.5 MiB [] 0% 126.2s172.5 MiB [] 0% 106.0s172.5 MiB [] 0% 88.7s172.5 MiB [] 0% 73.9s172.5 MiB [] 0% 62.2s172.5 MiB [] 0% 53.5s172.5 MiB [] 0% 43.4s172.5 MiB [] 0% 34.6s172.5 MiB [] 0% 30.6s172.5 MiB [] 1% 22.8s172.5 MiB [] 1% 18.2s172.5 MiB [] 1% 15.3s172.5 MiB [] 2% 11.3s172.5 MiB [] 3% 9.4s172.5 MiB [] 3% 8.4s172.5 MiB [] 4% 7.2s172.5 MiB [] 5% 7.0s172.5 MiB [] 6% 6.1s172.5 MiB [] 7% 5.2s172.5 MiB [] 7% 5.0s172.5 MiB [] 8% 4.8s172.5 MiB [] 9% 4.4s172.5 MiB [] 10% 4.2s172.5 MiB [] 11% 4.0s172.5 MiB [] 12% 3.6s172.5 MiB [] 12% 3.7s172.5 MiB [] 13% 3.4s172.5 MiB [] 14% 3.3s172.5 MiB [] 15% 3.1s172.5 MiB [] 16% 3.0s172.5 MiB [] 17% 2.9s172.5 MiB [] 18% 2.9s172.5 MiB [] 19% 2.8s172.5 MiB [] 20% 2.8s172.5 MiB [] 22% 2.6s172.5 MiB [] 23% 2.6s172.5 MiB [] 24% 2.5s172.5 MiB [] 25% 2.4s172.5 MiB [] 26% 2.4s172.5 MiB [] 27% 2.3s172.5 MiB [] 29% 2.2s172.5 MiB [] 31% 2.1s172.5 MiB [] 33% 2.0s172.5 MiB [] 34% 1

In [4]:
!ollama pull llama3.2:3b

In [7]:
!ollama pull nomic-embed-text:latest

In [5]:
!ollama list

NAME           ID              SIZE      MODIFIED               
llama3.2:3b    a80c4f17acd5    2.0 GB    Less than a second ago    


In [6]:
%%writefile app.py
import os
import ollama
import streamlit as st
from ddgs import DDGS
import asyncio
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser
import requests
from crawl4ai import AsyncWebCrawler, BrowserConfig, CacheMode, CrawlerRunConfig
from crawl4ai.content_filter_strategy import BM25ContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
from crawl4ai.models import CrawlResult
import chromadb
import tempfile
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb.config import Settings
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
system_prompt = """
You are an AI assistant tasked with providing detailed answers based solely on the given context.
Your goal is to analyze the information provided and formulate a comprehensive, well-structured response to the question.

Context will be passed as "Context:"
User question will be passed as "Question:"

To answer the question:
1. Thoroughly analyze the context, identifying key information relevant to the question.
2. Organize your thoughts and plan your response to ensure a logical flow of information.
3. Formulate a detailed answer that directly addresses the question, using only the information provided in the context.
4. When the context supports an answer, ensure your response is clear, concise, and directly addresses the question.
5. When there is no context, just say you have no context and stop immediately.
6. If the context doesn't contain sufficient information to fully answer the question, state this clearly in your response.
7. Avoid explaining why you cannot answer or speculating about missing details. Simply state that you lack sufficient context when necessary.

Format your response as follows:
1. Use clear, concise language.
2. Organize your answer into paragraphs for readability.
3. Use bullet points or numbered lists where appropriate to break down complex information.
4. If relevant, include any headings or subheadings to structure your response.
5. Ensure proper grammar, punctuation, and spelling throughout your answer.
6. Do not mention what you received in context, just focus on answering based on the context.

Important: Base your entire response solely on the information provided in the context. Do not include any external knowledge or assumptions not present in the given text.
"""


def call_llm(prompt: str, with_context: bool = True, context: str | None = None):
    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": f"Context: {context}, Question: {prompt}",
        },
    ]

    if not with_context:
        messages.pop(0)
        messages[0]["content"] = prompt

    response = ollama.chat(model="llama3.2:3b", stream=True, messages=messages)

    for chunk in response:
        if chunk["done"] is False:
            yield chunk["message"]["content"]
        else:
            break

def get_vector_collection() -> tuple[chromadb.Collection, chromadb.Client]:
    embedding_function = SentenceTransformerEmbeddingFunction(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    chroma_client = chromadb.PersistentClient(
        path="./web-search-llm-db",
        settings=Settings(anonymized_telemetry=False)
    )
    collection = chroma_client.get_or_create_collection(
        name="web_llm",
        embedding_function=embedding_function,
        metadata={"hnsw:space": "cosine"},
    )
    return (
        chroma_client.get_or_create_collection(
            name="web_llm",
            embedding_function=embedding_function,
            metadata={"hnsw:space": "cosine"},
        ),
        chroma_client,)


def normalize_url(url):
    normalized_url = (
        url.replace("https://", "")
        .replace("www.", "")
        .replace("/", "_")
        .replace("-", "_")
        .replace(".", "_")
    )
    print("Normalized URL", normalized_url)
    return normalized_url

def add_to_vector_database(results: list[CrawlResult]):
    collection, _ = get_vector_collection()

    for result in results:
        documents, metadatas, ids = [], [], []

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=400,
            chunk_overlap=100,
            separators=["\n\n", "\n", ".", "?", "!", " ", ""],
        )
        markdown_result = None
        if hasattr(result, "markdown_v2") and getattr(result.markdown_v2, "fit_markdown", None):
            markdown_result = result.markdown_v2.fit_markdown
        elif hasattr(result, "markdown") and result.markdown:
            markdown_result = result.markdown

        if not markdown_result:
            continue

        temp_file = tempfile.NamedTemporaryFile("w", suffix=".md", delete=False)
        temp_file.write(markdown_result)
        temp_file.flush()

        loader = UnstructuredMarkdownLoader(temp_file.name, mode="single")
        docs = loader.load()
        all_splits = text_splitter.split_documents(docs)
        os.unlink(temp_file.name)  # Delete the temporary file

        normalized_url = normalize_url(result.url)

        if all_splits:
            for idx, split in enumerate(all_splits):
                documents.append(split.page_content)
                metadatas.append({"source": result.url})
                ids.append(f"{normalized_url}_{idx}")

            print("Upsert collection: ", id(collection))
            collection.upsert(
                documents=documents,
                metadatas=metadatas,
                ids=ids,
            )


async def crawl_webpages(urls:list[str],prompt:str)->CrawlResult:
  bm25_filter=BM25ContentFilter(user_query=prompt,bm25_threshold=1.2)
  md_generator = DefaultMarkdownGenerator(content_filter=bm25_filter)
  crawler_config = CrawlerRunConfig(
        markdown_generator=md_generator,
        excluded_tags=["nav", "footer", "header", "form", "img", "a"],
        only_text=True,
        exclude_social_media_links=True,
        keep_data_attributes=False,
        cache_mode=CacheMode.BYPASS,
        remove_overlay_elements=True,
        user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
        page_timeout=20000,  # in ms: 20 seconds
    )
  browser_config = BrowserConfig(headless=True, text_mode=True, light_mode=True)
  async with AsyncWebCrawler(config=browser_config) as crawler:
        results = await crawler.arun_many(urls, config=crawler_config)
        return results

def check_robots_txt(urls:list[str])->list[str]:
  allowed_urls=[]
  for url in urls:
    try:
      robots_url=f"{urlparse(url=url).scheme}://{urlparse(url).netloc}/robots.txt"
      rp = RobotFileParser(robots_url)
      rp.read()

      if rp.can_fetch("*", url):
        allowed_urls.append(url)
    except:
      allowed_urls.append(url)

  return allowed_urls



def get_web_urls(search_term:str,num_result:int=10)->list[str]:
  try:
    discard_urls=["youtube.com", "britannica.com", "vimeo.com"]
    for url in discard_urls:
      search_term += f' -site:{url}'

    results = list(DDGS().text(search_term, max_results=num_result))

    urls = [result['href'] for result in results if 'href' in result]
    #st.write("Extracted URLs:", urls)

    return check_robots_txt(urls)
  except Exception as e:
    print(f"Failed to fetch results from the web", str(e))
    st.write(f"Failed to fetch results from the web", str(e))
    st.stop()


async def run():
    st.set_page_config(page_title="LLM with Web Search")

    st.header("🔍 LLM Web Search")
    prompt = st.text_area(
        label="Put your query here",
        placeholder="Add your query...",
        label_visibility="hidden",
    )
    is_web_search = st.toggle("Enable web search", value=False, key="enable_web_search")
    go = st.button(
        "⚡️ Go",
    )

    collection, chroma_client = get_vector_collection()

    if prompt and go:
        if is_web_search:
            web_urls = get_web_urls(search_term=prompt)
            if not web_urls:
                st.write("No results found.")
                st.stop()

            results = await crawl_webpages(urls=web_urls, prompt=prompt)
            add_to_vector_database(results)

            qresults = collection.query(query_texts=[prompt], n_results=10)
            context = qresults.get("documents")[0]

            chroma_client.delete_collection(
                name="web_llm"
            )  # Delete collection after use

            llm_response = call_llm(
                context=context, prompt=prompt, with_context=is_web_search
            )
            st.write_stream(llm_response)
        else:
            llm_response = call_llm(prompt=prompt, with_context=is_web_search)
            st.write_stream(llm_response)


if __name__ == "__main__":
    asyncio.run(run())

Writing app.py


In [7]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501, "http")
print("Streamlit App URL:", public_url)


Streamlit App URL: NgrokTunnel: "https://85914baa1e8f.ngrok-free.app" -> "http://localhost:8501"


In [8]:
!streamlit run app.py --server.port 8501 &>/dev/null&

In [22]:
!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: address already in use
